In [97]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import decoupler as dc
import plotnine as p9
import liana as li
import numpy as np

from pathlib import Path
from scipy.sparse import issparse, csr_matrix
from liana.method import cellphonedb, natmi
from mousipy import translate

import datetime

In [ ]:
#adata.obs.columns
#adata.var
#adata.layers['raw'] = adata.X.copy()
#sc.pp.normalize_total(adata, target_sum=1e4)
#sc.pp.log1p(adata)
#adata.layers['lognorm'] = adata.X.copy()
#just to look how it looks like, especially if we have Ensemble IDs or gene names
#adata.to_df(layer="raw")

#adata.var.index = adata.var["gene_ids"]

#control that the names are now symbols. If not, try adata.var_names = adata.var["gene_name"]

#adata.to_df(layer="raw")
#print(adata.var_names[:5])

# Merfish

In [93]:
##loading data

# nemelo by se najit hodne interakci
adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/data_true/merscope_final_annotated_subseted_gene_symbol.h5ad")
adata

adata.obs.columns
adata.var
adata.layers['raw'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers['lognorm'] = adata.X.copy()
#just to look how it looks like, especially if we have Ensemble IDs or gene names
adata.to_df(layer="raw")

adata.var.index = adata.var["gene_name"]

#control that the names are now symbols. If not, try adata.var_names = adata.var["gene_name"]

adata.to_df(layer="raw")
print(adata.var_names[:5])

print(li.mt.show_methods())
print(li.resource.show_resources())


##NATMI
print("START NATMI Merfish")
print(datetime.datetime.now())
resource_name = "mouseconsensus"  # Replace with the desired resource name if needed
lr_pairs = li.resource.select_resource(resource_name)
lr_pairs

def lr_pairs_in_adata(adata):
    genes_in_dataset = set(adata.var_names)  # Replace `adata.var_names` with your dataset's gene names if different
    
    # Filter the ligand-receptor pairs for those present in your dataset
    filtered_lr_pairs = lr_pairs[
        lr_pairs['ligand'].isin(genes_in_dataset) & lr_pairs['receptor'].isin(genes_in_dataset)
    ]
    
    return filtered_lr_pairs

filtered_lr_pairs = lr_pairs_in_adata(adata)

natmi(adata,
            groupby='cell_type', #or cell_type #or cell_labels
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='mouseconsensus',use_raw =False,
            expr_prop=0.1,layer = 'lognorm',
            verbose=True, key_added='natmi_res')

natmi_results = adata.uns['natmi_res']

natmi_results.to_csv("../output/natmi/natmi_results_merfish.csv", index=False)

print("END NATMI Merfish")
print(datetime.datetime.now())

Index(['Htr7', 'Gzmk', 'Arhgap36', 'Sema3c', 'Rxrg'], dtype='object', name='gene_name')
         Method Name Magnitude Score Specificity Score  \
0        CellPhoneDB        lr_means   cellphone_pvals   
0         Connectome       expr_prod     scaled_weight   
0             log2FC            None          lr_logfc   
0              NATMI       expr_prod       spec_weight   
0  SingleCellSignalR         lrscore              None   
0           CellChat        lr_probs    cellchat_pvals   
0     Rank_Aggregate  magnitude_rank  specificity_rank   
0     Geometric Mean       lr_gmeans       gmean_pvals   

                                           Reference  
0  Efremova, M., Vento-Tormo, M., Teichmann, S.A....  
0  Raredon, M.S.B., Yang, J., Garritano, J., Wang...  
0  Dimitrov, D., Türei, D., Garrido-Rodriguez, M....  
0  Hou, R., Denisenko, E., Ong, H.T., Ramilowski,...  
0  Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ...  
0  Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha...  
0  D

/home/blizkovs/.local/lib/python3.12/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/home/blizkovs/.local/lib/python3.12/site-packages/liana/method/_pipe_utils/_pre.py:150: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


0.83 of entities in the resource are missing from the data.
Generating ligand-receptor stats for 43488 samples and 1122 features
END NATMI Merfish
2025-04-28 22:05:38.223274


In [52]:
adata.var

,gene_ids,feature_types,genome
A1cf,ENSMUSG00000052595,Gene Expression,Unknown
A2m,ENSMUSG00000030111,Gene Expression,Unknown
Aatf,ENSMUSG00000018697,Gene Expression,Unknown
Abca1,ENSMUSG00000015243,Gene Expression,Unknown
Abca13,ENSMUSG00000004668,Gene Expression,Unknown
...,...,...,...
Zswim9,ENSMUSG00000070814,Gene Expression,Unknown
Zup1,ENSMUSG00000039531,Gene Expression,Unknown
Zyx,ENSMUSG00000029860,Gene Expression,Unknown
Zzef1,ENSMUSG00000055670,Gene Expression,Unknown


# Xenium

In [94]:
##loading data

adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/data_true/adata_xenium_final_annotated.h5ad")
adata

##NATMI
print("START NATMI Xenium")
print(datetime.datetime.now())
resource_name = "mouseconsensus"  # Replace with the desired resource name if needed
lr_pairs = li.resource.select_resource(resource_name)
lr_pairs

def lr_pairs_in_adata(adata):
    genes_in_dataset = set(adata.var_names)  # Replace `adata.var_names` with your dataset's gene names if different
    
    # Filter the ligand-receptor pairs for those present in your dataset
    filtered_lr_pairs = lr_pairs[
        lr_pairs['ligand'].isin(genes_in_dataset) & lr_pairs['receptor'].isin(genes_in_dataset)
    ]
    
    return filtered_lr_pairs

filtered_lr_pairs = lr_pairs_in_adata(adata)

natmi(adata,
            groupby='cell_groups', #or cell_type #or cell_labels
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='mouseconsensus',use_raw =False,
            expr_prop=0.1, layer = 'lognorm',
            verbose=True, key_added='natmi_res')

natmi_results = adata.uns['natmi_res']

natmi_results.to_csv("../output/natmi/natmi_results_xenium.csv", index=False)
print("END NATMI Xenium")
print(datetime.datetime.now())

START NATMI Xenium
2025-04-28 22:05:51.713380
Using the `lognorm` layer!
778 features of mat are empty, they will be removed.
The following cell identities were excluded: Serpinf1, Sncg


/home/blizkovs/.local/lib/python3.12/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/home/blizkovs/.local/lib/python3.12/site-packages/pandas/core/indexing.py:1857: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
/home/blizkovs/.local/lib/python3.12/site-packages/liana/method/_pipe_utils/_pre.py:150: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


0.51 of entities in the resource are missing from the data.
Generating ligand-receptor stats for 2292 samples and 4228 features
END NATMI Xenium
2025-04-28 22:06:02.074689


# Cosmx

In [103]:
adata.var_names

Index(['Chrna4', 'Slc6a1', 'Cd109', 'Ldha', 'Aldoc', 'Drd1', 'Tank', 'Rit2',
       'Prkag2', 'Lpar3',
       ...
       'NegPrb1', 'NegPrb2', 'NegPrb3', 'NegPrb4', 'NegPrb5', 'NegPrb6',
       'NegPrb7', 'NegPrb8', 'NegPrb9', 'NegPrb10'],
      dtype='object', length=960)

In [101]:
##loading data

adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/data_true/cosmx_final_annotated.h5ad")
adata

##NATMI
print("START NATMI Cosmx")
print(datetime.datetime.now())
resource_name = "mouseconsensus"  # Replace with the desired resource name if needed
lr_pairs = li.resource.select_resource(resource_name)
lr_pairs

def lr_pairs_in_adata(adata):
    genes_in_dataset = set(adata.var_names)  # Replace `adata.var_names` with your dataset's gene names if different
    
    # Filter the ligand-receptor pairs for those present in your dataset
    filtered_lr_pairs = lr_pairs[
        lr_pairs['ligand'].isin(genes_in_dataset) & lr_pairs['receptor'].isin(genes_in_dataset)
    ]
    
    return filtered_lr_pairs

filtered_lr_pairs = lr_pairs_in_adata(adata)

natmi(adata,
            groupby='cell_groups', #or cell_type #or cell_labels
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='mouseconsensus',use_raw =False,
            expr_prop=0.1,layer = 'lognorm',
            verbose=True, key_added='natmi_res')

natmi_results = adata.uns['natmi_res']

natmi_results.to_csv("../output/natmi/natmi_results_cosmx.csv", index=False)
print("END NATMI Cosmx")
print(datetime.datetime.now())

START NATMI Cosmx
2025-04-30 10:31:30.396255
Using the `lognorm` layer!
7 samples of mat are empty, they will be removed.


/home/blizkovs/.local/lib/python3.12/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/home/blizkovs/.local/lib/python3.12/site-packages/pandas/core/indexing.py:1857: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
/home/blizkovs/.local/lib/python3.12/site-packages/liana/method/_pipe_utils/_pre.py:150: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


0.83 of entities in the resource are missing from the data.
Generating ligand-receptor stats for 48549 samples and 960 features
END NATMI Cosmx
2025-04-30 10:31:37.929089


# Visium

In [117]:
##chybejici symbol

adata = sc.read_h5ad("../PRIMUS/data/33_data/transfer/Sofie_bc_prace/visium_hd_annotated.h5ad")
adata


/home/blizkovs/.local/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.


AnnData object with n_obs × n_vars = 273789 × 16797
    obs: 'in_tissue', 'array_row', 'array_col', 'location_id', 'region', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'barcode', 'label'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'label_colors', 'log1p', 'spatialdata_attrs'
    obsm: 'spatial'
    layers: 'lognorm'

In [120]:
adata.var_names

Index(['Xkr4', 'Rp1', 'Sox17', 'Lypla1', 'Tcea1', 'Rgs20', 'Atp6v1h', 'Oprk1',
       'Npbwr1', 'Rb1cc1',
       ...
       'mt-Atp8', 'mt-Atp6', 'mt-Co3', 'mt-Nd3', 'mt-Nd4l', 'mt-Nd4', 'mt-Nd5',
       'mt-Nd6', 'mt-Cytb', 'Vamp7'],
      dtype='object', length=16797)

In [119]:
##NATMI
print("START NATMI Visium")
print(datetime.datetime.now())
resource_name = "mouseconsensus"  # Replace with the desired resource name if needed
lr_pairs = li.resource.select_resource(resource_name)
lr_pairs

def lr_pairs_in_adata(adata):
    genes_in_dataset = set(adata.var_names)  # Replace `adata.var_names` with your dataset's gene names if different
    
    # Filter the ligand-receptor pairs for those present in your dataset
    filtered_lr_pairs = lr_pairs[
        lr_pairs['ligand'].isin(genes_in_dataset) & lr_pairs['receptor'].isin(genes_in_dataset)
    ]
    
    return filtered_lr_pairs

filtered_lr_pairs = lr_pairs_in_adata(adata)

natmi(adata,
            groupby='label', #or cell_type #or cell_labels
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='mouseconsensus', use_raw = False,
            expr_prop=0.1,layer = 'lognorm',
            verbose=True, key_added='natmi_res')

natmi_results = adata.uns['natmi_res']

natmi_results.to_csv("../output/natmi/natmi_results_visiumhd.csv", index=False)
print("END NATMI Visium")
print(datetime.datetime.now())

START NATMI Visium
2025-04-30 10:47:06.217371
Using the `lognorm` layer!


/home/blizkovs/.local/lib/python3.12/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/home/blizkovs/.local/lib/python3.12/site-packages/liana/method/_pipe_utils/_pre.py:150: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


0.13 of entities in the resource are missing from the data.
Generating ligand-receptor stats for 273789 samples and 16797 features
END NATMI Visium
2025-04-30 10:47:28.383605
